In [10]:
from sklearn.model_selection import train_test_split
from mtcnn.mtcnn import MTCNN
from sklearn.preprocessing import Normalizer , LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from keras_facenet import FaceNet
import numpy as np
import wget
import cv2 as cv
import os

In [2]:
IMAGES_PATH = os.path.join("images")
CROPPED_IMAGE_PATH = os.path.join("images" , "cropped_images")
MODEL_PATH = os.path.join("facenet")

In [3]:

if os.path.exists(CROPPED_IMAGE_PATH):
    labels = {"_".join(label.split("_")[1:]):i for i , label in enumerate(os.listdir(CROPPED_IMAGE_PATH))}
print(labels)
X = []
Y = []

{'benedict_cumberbatch': 0, 'chris_evans': 1, 'chris_hemsworth': 2, 'robert_downey': 3, 'tom_hiddleston': 4}


In [4]:
for label , index in labels.items():
    path = os.path.join(CROPPED_IMAGE_PATH , f"cropped_{label}")
    if os.path.exists(path):
        images = os.listdir(path)
        for image in images:
            X.append(cv.imread(os.path.join(path , image)))
            Y.append(index)
X = np.asarray(X)
Y = np.asarray(Y)      
img = cv.imread("deepika(1).pdf")
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , train_size=0.8, shuffle=True)      

C:\Users\parij\AppData\Local\Temp\ipykernel_21524\988532183.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)


In [5]:
# def preprocessing(X):
#     X = np.resize(X , (160 , 160 , 3))
#     X = X.astype("float32")
#     mean , std = X.mean() , X.std()
#     X = (X - mean)/std
#     return X

def get_embedding(model , X):
    samples = np.expand_dims(X , axis=0)
    return model.embeddings(samples)[0]

In [6]:
facenet_model = FaceNet()
new_trainX = []
new_testX = []
for pixels in X_train:
    new_trainX.append(get_embedding(facenet_model , pixels))
for pixels in X_test:
    new_testX.append(get_embedding(facenet_model , pixels))
X_train = np.asarray(new_trainX)
X_test = np.asarray(new_testX)
print("Training Shape: " , str(X_train.shape))
print("Testing Shape: " , str(X_test.shape))

Training Shape:  (49, 512)
Testing Shape:  (13, 512)


In [7]:
in_encoder = Normalizer('l2')
out_encoder = LabelEncoder()
X_train = in_encoder.transform(X_train)
X_test = in_encoder.transform(X_test)

In [8]:
model = SVC(kernel="linear")
model.fit(X_train , Y_train)

SVC(kernel='linear')

In [9]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)
score_train = accuracy_score(train_pred , Y_train)
score_test = accuracy_score(test_pred , Y_test)
print("training score: " , str(score_train))
print("testing score: " , str(score_test))

training score:  1.0
testing score:  1.0


In [11]:
img_path = "images\\tom-hiddleston-loki.jpg"
detector = MTCNN()
result = detector.detect_faces(cv.imread(img_path))
print(result)

InvalidImage: Image not valid.